In [15]:
from surprise.prediction_algorithms import SVD
import pandas as pd
df = pd.read_csv('../../../data/ratings.csv')
df = df.drop(columns='timestamp')
movies = pd.read_csv('../../../data/movies.csv')

In [16]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(df, reader)
dataset = data.build_full_trainset()

In [17]:
svd = SVD(n_factors=50, reg_all=0.05)


In [18]:
def movie_rater(movies, num=5, genre=None):
    userID = 0
    rating_list = []
    while num > 0:
        if genre:
            movie = movies[movies['genres'].str.contains(genre)].sample(1)
        else:
            movie = movies.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      

In [31]:
new_ratings_df = df.append(movie_rater(movies), ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

      movieId                                     title          genres
8437   111743  A Million Ways to Die in the West (2014)  Comedy|Western


How do you rate this movie on a scale of 1-5, press n if you have not seen :
 5


      movieId                       title genres
4279     6246  Talent for the Game (1991)  Drama


How do you rate this movie on a scale of 1-5, press n if you have not seen :
 4


      movieId                   title                 genres
1616     2160  Rosemary's Baby (1968)  Drama|Horror|Thriller


How do you rate this movie on a scale of 1-5, press n if you have not seen :
 3


      movieId               title        genres
7830    93114  Ro.Go.Pa.G. (1963)  Comedy|Drama


How do you rate this movie on a scale of 1-5, press n if you have not seen :
 2


      movieId                    title       genres
7362    78746  Best Worst Movie (2009)  Documentary


How do you rate this movie on a scale of 1-5, press n if you have not seen :
 1


In [32]:
svd.fit(new_data.build_full_trainset())

In [33]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in df['movieId'].unique():
    list_of_movies.append((m_id, svd.predict(1000,m_id)[3]))

In [34]:
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)


In [35]:
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies, movies, 5)

Recommendation #  1 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  2 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  3 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  4 :  2226    Fight Club (1999)
Name: title, dtype: object 

Recommendation #  5 :  841    Streetcar Named Desire, A (1951)
Name: title, dtype: object 

